In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import chardet

# **Load the Data**

In [3]:
# Load Excel file
file_path = r'f1_pitstops_2018_2024.xlsx'
df = pd.read_excel(file_path, sheet_name='f1_pitstops_2018_2024')
display(df.head())

,Season,Round,Circuit,Driver,Constructor,Laps,Position,TotalPitStops,AvgPitStopTime,Race Name,...,Fast Lap Attempts,Position Changes,Driver Aggression Score,Abbreviation,Stint,Tire Compound,Stint Length,Pit_Lap,Pit_Time,tire compound
0,2018,1,Albert Park Grand Prix Circuit,Sebastian Vettel,Ferrari,58,1,1,21.787,Australian Grand Prix,...,44.76882,0.000000,6.755003,VET,1.0,ULTRASOFT,25.0,26.0,21.787,C4
1,2018,1,Albert Park Grand Prix Circuit,Sebastian Vettel,Ferrari,58,1,1,21.787,Australian Grand Prix,...,44.76882,0.000000,6.755003,VET,2.0,SOFT,32.0,NaN,Final Stint,C5
2,2018,1,Albert Park Grand Prix Circuit,Lewis Hamilton,Mercedes,58,2,1,21.821,Australian Grand Prix,...,44.73482,0.043478,6.754254,HAM,1.0,ULTRASOFT,17.0,19.0,21.821,C4
3,2018,1,Albert Park Grand Prix Circuit,Lewis Hamilton,Mercedes,58,2,1,21.821,Australian Grand Prix,...,44.73482,0.043478,6.754254,HAM,2.0,SOFT,39.0,NaN,Final Stint,C5
4,2018,1,Albert Park Grand Prix Circuit,Kimi RÃƒÆ’Ã‚Â¤ikkÃƒÆ’Ã‚Â¶nen,Ferrari,58,3,1,21.421,Australian Grand Prix,...,45.13482,0.086957,6.818562,RAI,1.0,ULTRASOFT,17.0,18.0,21.421,C4


# **Check Data Types**

In [4]:
# 1. Check data types and columns
type_of_column = [i for i in df.dtypes]
column = [j for j in df.columns]
info = pd.concat([pd.Series(type_of_column), pd.Series(column)], axis=1)
info.columns = ['DataType', 'ColumnName']
display(info)

,DataType,ColumnName
0,int64,Season
1,int64,Round
2,object,Circuit
3,object,Driver
4,object,Constructor
5,int64,Laps
6,int64,Position
7,int64,TotalPitStops
8,float64,AvgPitStopTime
9,object,Race Name


In [5]:
# Convert categorical columns
categorical_cols = ['Circuit','Driver','Constructor','Race Name','Location','Country','Abbreviation','Tire Compound','tire compound ']
for col in categorical_cols:
    df[col] = df[col].astype('category')

In [23]:
df['date'] = pd.to_datetime(df['date'], errors='coerce', dayfirst=True)

df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

df.drop(columns=['date'], inplace=True)

print(" 'Date' column successfully split into 'day', 'month', and 'year'.")

display(df[['season', 'round', 'year', 'month', 'day']].head())

 'Date' column successfully split into 'day', 'month', and 'year'.


,season,round,year,month,day
0,2018,1,2018,3,25
1,2018,1,2018,3,25
2,2018,1,2018,3,25
3,2018,1,2018,3,25
4,2018,1,2018,3,25


In [24]:
date_cols_to_convert = ['year', 'month', 'day']

for col in date_cols_to_convert:
    df[col] = df[col].astype(int)


print("\nData types of the date columns")
print(df[['year', 'month', 'day']].dtypes)


Data types of the date columns
year     int64
month    int64
day      int64
dtype: object


In [6]:
# remove Z, then parse as time
df['Time_of_race'] = (
    df['Time_of_race']
    .astype(str)  # ensure string
    .str.replace('Z','', regex=False)  # remove Z
)
df['Time_of_race'] = pd.to_datetime(
    df['Time_of_race'],
    format='%H:%M:%S',
    errors='coerce'
).dt.time

In [7]:
# Convert numeric columns
numeric_cols = ['Season','Round','Laps','Position','TotalPitStops','Stint','Stint Length','Pit_Lap']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [8]:
# Convert to string first to handle both cases uniformly
df['Pit_Time'] = df['Pit_Time'].astype(str)
# Replace 'Final Stint' with a sentinel numeric value (e.g. -1)
df['Pit_Time'] = df['Pit_Time'].replace('Final Stint', -1)
# Convert the rest to numeric (float)
df['Pit_Time'] = pd.to_numeric(df['Pit_Time'], errors='coerce')

In [9]:
# Check the result
df.dtypes

,0
Season,int64
Round,int64
Circuit,category
Driver,category
Constructor,category
Laps,int64
Position,int64
TotalPitStops,int64
AvgPitStopTime,float64
Race Name,category


# **Renaming Columns**

In [10]:
df.columns = (
    df.columns
    .str.strip()            # remove spaces
    .str.lower()            # lowercase everything
    .str.replace(' ', '_')  # replace spaces with underscores
)


print(df.columns.tolist())

['season', 'round', 'circuit', 'driver', 'constructor', 'laps', 'position', 'totalpitstops', 'avgpitstoptime', 'race_name', 'date', 'time_of_race', 'location', 'country', 'air_temp_c', 'track_temp_c', 'humidity_%', 'wind_speed_kmh', 'lap_time_variation', 'total_pit_stops', 'tire_usage_aggression', 'fast_lap_attempts', 'position_changes', 'driver_aggression_score', 'abbreviation', 'stint', 'tire_compound', 'stint_length', 'pit_lap', 'pit_time', 'tire_compound']


In [11]:
column_list = df.columns.tolist()

column_list[-1] = 'tire_compound_code'

df.columns = column_list

print(df.columns.tolist())

['season', 'round', 'circuit', 'driver', 'constructor', 'laps', 'position', 'totalpitstops', 'avgpitstoptime', 'race_name', 'date', 'time_of_race', 'location', 'country', 'air_temp_c', 'track_temp_c', 'humidity_%', 'wind_speed_kmh', 'lap_time_variation', 'total_pit_stops', 'tire_usage_aggression', 'fast_lap_attempts', 'position_changes', 'driver_aggression_score', 'abbreviation', 'stint', 'tire_compound', 'stint_length', 'pit_lap', 'pit_time', 'tire_compound_code']


# **Dropping Columns**

In [12]:
df.drop('total_pit_stops', axis=1, inplace=True)

# (Optional) Verify that the column has been removed
print("Column 'total_pit_stops' has been successfully dropped.")
print("\nUpdated list of columns:")
print(df.columns)

Column 'total_pit_stops' has been successfully dropped.

Updated list of columns:
Index(['season', 'round', 'circuit', 'driver', 'constructor', 'laps',
       'position', 'totalpitstops', 'avgpitstoptime', 'race_name', 'date',
       'time_of_race', 'location', 'country', 'air_temp_c', 'track_temp_c',
       'humidity_%', 'wind_speed_kmh', 'lap_time_variation',
       'tire_usage_aggression', 'fast_lap_attempts', 'position_changes',
       'driver_aggression_score', 'abbreviation', 'stint', 'tire_compound',
       'stint_length', 'pit_lap', 'pit_time', 'tire_compound_code'],
      dtype='object')


# **Fixing mojibake**

In [13]:
def find_non_ascii(text):
  return not all(ord(char) < 128 for char in text)

# Get all unique circuit names from the column
unique_circuits = df['circuit'].unique()

# Filter the list to find only those with non-ASCII characters
problematic_circuits = [circuit for circuit in unique_circuits if find_non_ascii(str(circuit))]

if problematic_circuits:
  print("Found potential encoding issues in the following circuit names:")
  for circuit in problematic_circuits:
    print(circuit)
else:
  print("No non-ASCII characters found in the 'Circuit' column.")

Found potential encoding issues in the following circuit names:
AutÃƒÆ’Ã‚Â³dromo Hermanos RodrÃƒÆ’Ã‚Â­guez
NÃƒÆ’Ã‚Â¼rburgring
AutÃƒÆ’Ã‚Â³dromo Internacional do Algarve
AutÃƒÆ’Ã‚Â³dromo JosÃƒÆ’Ã‚Â© Carlos Pace


In [14]:
# key = 'incorrect name', value = 'correct name'

name_corrections = {
    'AutÃƒÆ’Ã‚Â³dromo Hermanos RodrÃƒÆ’Ã‚Â­guez': 'Autódromo Hermanos Rodríguez',
    'NÃƒÆ’Ã‚Â¼rburgring': 'Nürburgring',
    'AutÃƒÆ’Ã‚Â³dromo Internacional do Algarve': 'Autódromo Internacional do Algarve',
    'AutÃƒÆ’Ã‚Â³dromo JosÃƒÆ’Ã‚Â© Carlos Pace': 'Autódromo José Carlos Pace'
}

# rename the categories
df['circuit'] = df['circuit'].cat.rename_categories(name_corrections)

print("Categories have been successfully renamed.")
print(df['circuit'].unique())

Categories have been successfully renamed.
['Albert Park Grand Prix Circuit', 'Bahrain International Circuit', 'Shanghai International Circuit', 'Baku City Circuit', 'Circuit de Barcelona-Catalunya', ..., 'Losail International Circuit', 'Jeddah Corniche Circuit', 'Miami International Autodrome', 'Las Vegas Strip Street Circuit', 'Autódromo José Carlos Pace']
Length: 31
Categories (31, object): ['Albert Park Grand Prix Circuit', 'Autodromo Enzo e Dino Ferrari',
                          'Autodromo Internazionale del Mugello', 'Autodromo Nazionale di Monza', ...,
                          'Silverstone Circuit', 'Sochi Autodrom', 'Suzuka Circuit', 'Yas Marina Circuit']


In [15]:
def find_non_ascii(text):
  return not all(ord(char) < 128 for char in text)

# Get all unique driver names from the column
unique_drivers = df['driver'].unique()

problematic_drivers = [driver for driver in unique_drivers if find_non_ascii(driver)]

if problematic_drivers:
  print("Found potential encoding issues in the following driver names:")
  for driver in problematic_drivers:
    print(f"- {driver}")
else:
  print("No names with garbled encoding found in the 'driver' column.")

Found potential encoding issues in the following driver names:
- Kimi RÃƒÆ’Ã‚Â¤ikkÃƒÆ’Ã‚Â¶nen
- Nico HÃƒÆ’Ã‚Â¼lkenberg
- Sergio PÃƒÆ’Ã‚Â©rez


In [16]:
driver_name_corrections = {
    'Kimi RÃƒÆ’Ã‚Â¤ikkÃƒÆ’Ã‚Â¶nen': 'Kimi Raikkönen',
    'Nico HÃƒÆ’Ã‚Â¼lkenberg': 'Nico Hülkenberg',
    'Sergio PÃƒÆ’Ã‚Â©rez': 'Sergio Pérez'
}


df['driver'] = df['driver'].cat.rename_categories(driver_name_corrections)

# --- Verification Step ---
print("✅ Categories have been successfully renamed.")
print("\nVerifying the unique driver names after correction:")
print(df['driver'].unique())

✅ Categories have been successfully renamed.

Verifying the unique driver names after correction:
['Sebastian Vettel', 'Lewis Hamilton', 'Kimi Raikkönen', 'Daniel Ricciardo', 'Fernando Alonso', ..., 'Oscar Piastri', 'Liam Lawson', 'Oliver Bearman', 'Franco Colapinto', 'Jack Doohan']
Length: 40
Categories (40, object): ['Alexander Albon', 'Antonio Giovinazzi', 'Brendon Hartley', 'Carlos Sainz',
                          ..., 'Sergio Pérez', 'Stoffel Vandoorne', 'Valtteri Bottas',
                          'Yuki Tsunoda']


# **Missing values analysis**

In [17]:
missing_values = df.isnull().sum().sort_values(ascending=False)
percentage_of_missing = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({'Missing Values': missing_values, 'Percentage': percentage_of_missing})
display(missing_df)

,Missing Values,Percentage
pit_lap,2810,38.106862
location,373,5.058313
date,373,5.058313
time_of_race,373,5.058313
race_name,373,5.058313
country,373,5.058313
track_temp_c,373,5.058313
humidity_%,373,5.058313
air_temp_c,373,5.058313
wind_speed_kmh,373,5.058313


In [18]:
# avgpitstoptime

# Filter for the rows that are real missing data
true_missing_values = df[(df['totalpitstops'] > 0) & (df['avgpitstoptime'].isna())]

print(f"There are {len(true_missing_values)} rows with missing avgpitstoptime despite having pit stops.")

display(true_missing_values.head())

There are 0 rows with missing avgpitstoptime despite having pit stops.


,season,round,circuit,driver,constructor,laps,position,totalpitstops,avgpitstoptime,race_name,...,fast_lap_attempts,position_changes,driver_aggression_score,abbreviation,stint,tire_compound,stint_length,pit_lap,pit_time,tire_compound_code


In [19]:
# Set the pandas option to display all rows
pd.set_option('display.max_rows', None)

# Select the columns to group by
race_list = df[['season', 'round', 'circuit']]

# Drop duplicate rows to get a unique list of races
unique_races = race_list.drop_duplicates()

# Sort the results for a clean, chronological view
sorted_races = unique_races.sort_values(by=['season', 'round'])

print("--- The Complete List of Unique Races ---")
display(sorted_races)

# (Optional) Reset the display option back to the default
pd.reset_option('display.max_rows')

--- The Complete List of Unique Races ---


,season,round,circuit
0,2018,1,Albert Park Grand Prix Circuit
36,2018,2,Bahrain International Circuit
85,2018,3,Shanghai International Circuit
129,2018,4,Baku City Circuit
176,2018,5,Circuit de Barcelona-Catalunya
213,2018,6,Circuit de Monaco
254,2018,7,Circuit Gilles Villeneuve
290,2018,8,Circuit Paul Ricard
324,2018,9,Red Bull Ring
363,2018,10,Silverstone Circuit


In [20]:
manual_race_lookup = {
    (2018, 1, 'Albert Park Grand Prix Circuit'): 'Australian Grand Prix',
    (2018, 2, 'Bahrain International Circuit'): 'Bahrain Grand Prix',
    (2018, 3, 'Shanghai International Circuit'): 'Chinese Grand Prix',
    (2018, 4, 'Baku City Circuit'): 'Azerbaijan Grand Prix',
    (2018, 5, 'Circuit de Barcelona-Catalunya'): 'Spanish Grand Prix',
    (2018, 6, 'Circuit de Monaco'): 'Monaco Grand Prix',
    (2018, 7, 'Circuit Gilles Villeneuve'): 'Canadian Grand Prix',
    (2018, 8, 'Circuit Paul Ricard'): 'French Grand Prix',
    (2018, 9, 'Red Bull Ring'): 'Austrian Grand Prix',
    (2018, 10, 'Silverstone Circuit'): 'British Grand Prix',
    (2018, 11, 'Hockenheimring'): 'German Grand Prix',
    (2018, 12, 'Hungaroring'): 'Hungarian Grand Prix',
    (2018, 13, 'Circuit de Spa-Francorchamps'): 'Belgian Grand Prix',
    (2018, 14, 'Autodromo Nazionale di Monza'): 'Italian Grand Prix',
    (2018, 15, 'Marina Bay Street Circuit'): 'Singapore Grand Prix',
    (2018, 16, 'Sochi Autodrom'): 'Russian Grand Prix',
    (2018, 17, 'Suzuka Circuit'): 'Japanese Grand Prix',
    (2018, 18, 'Circuit of the Americas'): 'United States Grand Prix',
    (2018, 19, 'Autódromo Hermanos Rodríguez'): 'Mexican Grand Prix',
    (2018, 21, 'Yas Marina Circuit'): 'Abu Dhabi Grand Prix',
    (2019, 1, 'Albert Park Grand Prix Circuit'): 'Australian Grand Prix',
    (2019, 2, 'Bahrain International Circuit'): 'Bahrain Grand Prix',
    (2019, 3, 'Shanghai International Circuit'): 'Chinese Grand Prix',
    (2019, 4, 'Baku City Circuit'): 'Azerbaijan Grand Prix',
    (2019, 5, 'Circuit de Barcelona-Catalunya'): 'Spanish Grand Prix',
    (2019, 6, 'Circuit de Monaco'): 'Monaco Grand Prix',
    (2019, 7, 'Circuit Gilles Villeneuve'): 'Canadian Grand Prix',
    (2019, 8, 'Circuit Paul Ricard'): 'French Grand Prix',
    (2019, 9, 'Red Bull Ring'): 'Austrian Grand Prix',
    (2019, 10, 'Silverstone Circuit'): 'British Grand Prix',
    (2019, 11, 'Hockenheimring'): 'German Grand Prix',
    (2019, 12, 'Hungaroring'): 'Hungarian Grand Prix',
    (2019, 13, 'Circuit de Spa-Francorchamps'): 'Belgian Grand Prix',
    (2019, 14, 'Autodromo Nazionale di Monza'): 'Italian Grand Prix',
    (2019, 15, 'Marina Bay Street Circuit'): 'Singapore Grand Prix',
    (2019, 16, 'Sochi Autodrom'): 'Russian Grand Prix',
    (2019, 17, 'Suzuka Circuit'): 'Japanese Grand Prix',
    (2019, 18, 'Autódromo Hermanos Rodríguez'): 'Mexican Grand Prix',
    (2019, 19, 'Circuit of the Americas'): 'United States Grand Prix',
    (2019, 21, 'Yas Marina Circuit'): 'Abu Dhabi Grand Prix',
    (2020, 1, 'Red Bull Ring'): 'Austrian Grand Prix',
    (2020, 2, 'Red Bull Ring'): 'Styrian Grand Prix',
    (2020, 3, 'Hungaroring'): 'Hungarian Grand Prix',
    (2020, 4, 'Silverstone Circuit'): 'British Grand Prix',
    (2020, 5, 'Silverstone Circuit'): '70th Anniversary Grand Prix',
    (2020, 6, 'Circuit de Barcelona-Catalunya'): 'Spanish Grand Prix',
    (2020, 7, 'Circuit de Spa-Francorchamps'): 'Belgian Grand Prix',
    (2020, 8, 'Autodromo Nazionale di Monza'): 'Italian Grand Prix',
    (2020, 9, 'Autodromo Internazionale del Mugello'): 'Tuscan Grand Prix',
    (2020, 10, 'Sochi Autodrom'): 'Russian Grand Prix',
    (2020, 11, 'Nürburgring'): 'Eifel Grand Prix',
    (2020, 12, 'Autódromo Internacional do Algarve'): 'Portuguese Grand Prix',
    (2020, 13, 'Autodromo Enzo e Dino Ferrari'): 'Emilia Romagna Grand Prix',
    (2020, 14, 'Istanbul Park'): 'Turkish Grand Prix',
    (2020, 15, 'Bahrain International Circuit'): 'Bahrain Grand Prix',
    (2020, 16, 'Bahrain International Circuit'): 'Sakhir Grand Prix',
    (2020, 17, 'Yas Marina Circuit'): 'Abu Dhabi Grand Prix',
    (2021, 1, 'Bahrain International Circuit'): 'Bahrain Grand Prix',
    (2021, 2, 'Autodromo Enzo e Dino Ferrari'): 'Emilia Romagna Grand Prix',
    (2021, 3, 'Autódromo Internacional do Algarve'): 'Portuguese Grand Prix',
    (2021, 4, 'Circuit de Barcelona-Catalunya'): 'Spanish Grand Prix',
    (2021, 5, 'Circuit de Monaco'): 'Monaco Grand Prix',
    (2021, 6, 'Baku City Circuit'): 'Azerbaijan Grand Prix',
    (2021, 7, 'Circuit Paul Ricard'): 'French Grand Prix',
    (2021, 8, 'Red Bull Ring'): 'Styrian Grand Prix',
    (2021, 9, 'Red Bull Ring'): 'Austrian Grand Prix',
    (2021, 10, 'Silverstone Circuit'): 'British Grand Prix',
    (2021, 11, 'Hungaroring'): 'Hungarian Grand Prix',
    (2021, 12, 'Circuit de Spa-Francorchamps'): 'Belgian Grand Prix',
    (2021, 13, 'Circuit Park Zandvoort'): 'Dutch Grand Prix',
    (2021, 14, 'Autodromo Nazionale di Monza'): 'Italian Grand Prix',
    (2021, 15, 'Sochi Autodrom'): 'Russian Grand Prix',
    (2021, 16, 'Istanbul Park'): 'Turkish Grand Prix',
    (2021, 17, 'Circuit of the Americas'): 'United States Grand Prix',
    (2021, 20, 'Losail International Circuit'): 'Qatar Grand Prix',
    (2021, 21, 'Jeddah Corniche Circuit'): 'Saudi Arabian Grand Prix',
    (2021, 22, 'Yas Marina Circuit'): 'Abu Dhabi Grand Prix',
    (2022, 1, 'Bahrain International Circuit'): 'Bahrain Grand Prix',
    (2022, 2, 'Jeddah Corniche Circuit'): 'Saudi Arabian Grand Prix',
    (2022, 3, 'Albert Park Grand Prix Circuit'): 'Australian Grand Prix',
    (2022, 4, 'Autodromo Enzo e Dino Ferrari'): 'Emilia Romagna Grand Prix',
    (2022, 5, 'Miami International Autodrome'): 'Miami Grand Prix',
    (2022, 6, 'Circuit de Barcelona-Catalunya'): 'Spanish Grand Prix',
    (2022, 7, 'Circuit de Monaco'): 'Monaco Grand Prix',
    (2022, 8, 'Baku City Circuit'): 'Azerbaijan Grand Prix',
    (2022, 9, 'Circuit Gilles Villeneuve'): 'Canadian Grand Prix',
    (2022, 10, 'Silverstone Circuit'): 'British Grand Prix',
    (2022, 11, 'Red Bull Ring'): 'Austrian Grand Prix',
    (2022, 12, 'Circuit Paul Ricard'): 'French Grand Prix',
    (2022, 13, 'Hungaroring'): 'Hungarian Grand Prix',
    (2022, 14, 'Circuit de Spa-Francorchamps'): 'Belgian Grand Prix',
    (2022, 15, 'Circuit Park Zandvoort'): 'Dutch Grand Prix',
    (2022, 16, 'Autodromo Nazionale di Monza'): 'Italian Grand Prix',
    (2022, 17, 'Marina Bay Street Circuit'): 'Singapore Grand Prix',
    (2022, 18, 'Suzuka Circuit'): 'Japanese Grand Prix',
    (2022, 19, 'Circuit of the Americas'): 'United States Grand Prix',
    (2022, 20, 'Autódromo Hermanos Rodríguez'): 'Mexico City Grand Prix',
    (2022, 22, 'Yas Marina Circuit'): 'Abu Dhabi Grand Prix',
    (2023, 1, 'Bahrain International Circuit'): 'Bahrain Grand Prix',
    (2023, 2, 'Jeddah Corniche Circuit'): 'Saudi Arabian Grand Prix',
    (2023, 3, 'Albert Park Grand Prix Circuit'): 'Australian Grand Prix',
    (2023, 4, 'Baku City Circuit'): 'Azerbaijan Grand Prix',
    (2023, 5, 'Miami International Autodrome'): 'Miami Grand Prix',
    (2023, 6, 'Circuit de Monaco'): 'Monaco Grand Prix',
    (2023, 7, 'Circuit de Barcelona-Catalunya'): 'Spanish Grand Prix',
    (2023, 8, 'Circuit Gilles Villeneuve'): 'Canadian Grand Prix',
    (2023, 9, 'Red Bull Ring'): 'Austrian Grand Prix',
    (2023, 10, 'Silverstone Circuit'): 'British Grand Prix',
    (2023, 11, 'Hungaroring'): 'Hungarian Grand Prix',
    (2023, 12, 'Circuit de Spa-Francorchamps'): 'Belgian Grand Prix',
    (2023, 13, 'Circuit Park Zandvoort'): 'Dutch Grand Prix',
    (2023, 14, 'Autodromo Nazionale di Monza'): 'Italian Grand Prix',
    (2023, 15, 'Marina Bay Street Circuit'): 'Singapore Grand Prix',
    (2023, 16, 'Suzuka Circuit'): 'Japanese Grand Prix',
    (2023, 17, 'Losail International Circuit'): 'Qatar Grand Prix',
    (2023, 18, 'Circuit of the Americas'): 'United States Grand Prix',
    (2023, 19, 'Autódromo Hermanos Rodríguez'): 'Mexico City Grand Prix',
    (2023, 21, 'Las Vegas Strip Street Circuit'): 'Las Vegas Grand Prix',
    (2023, 22, 'Yas Marina Circuit'): 'Abu Dhabi Grand Prix',
    (2024, 1, 'Bahrain International Circuit'): 'Bahrain Grand Prix',
    (2024, 2, 'Jeddah Corniche Circuit'): 'Saudi Arabian Grand Prix',
    (2024, 3, 'Albert Park Grand Prix Circuit'): 'Australian Grand Prix',
    (2024, 4, 'Suzuka Circuit'): 'Japanese Grand Prix',
    (2024, 5, 'Shanghai International Circuit'): 'Chinese Grand Prix',
    (2024, 6, 'Miami International Autodrome'): 'Miami Grand Prix',
    (2024, 7, 'Autodromo Enzo e Dino Ferrari'): 'Emilia Romagna Grand Prix',
    (2024, 8, 'Circuit de Monaco'): 'Monaco Grand Prix',
    (2024, 9, 'Circuit Gilles Villeneuve'): 'Canadian Grand Prix',
    (2024, 10, 'Circuit de Barcelona-Catalunya'): 'Spanish Grand Prix',
    (2024, 11, 'Red Bull Ring'): 'Austrian Grand Prix',
    (2024, 12, 'Silverstone Circuit'): 'British Grand Prix',
    (2024, 13, 'Hungaroring'): 'Hungarian Grand Prix',
    (2024, 14, 'Circuit de Spa-Francorchamps'): 'Belgian Grand Prix',
    (2024, 15, 'Circuit Park Zandvoort'): 'Dutch Grand Prix',
    (2024, 16, 'Autodromo Nazionale di Monza'): 'Italian Grand Prix',
    (2024, 17, 'Baku City Circuit'): 'Azerbaijan Grand Prix',
    (2024, 18, 'Marina Bay Street Circuit'): 'Singapore Grand Prix',
    (2024, 19, 'Circuit of the Americas'): 'United States Grand Prix',
    (2024, 20, 'Autódromo Hermanos Rodríguez'): 'Mexico City Grand Prix',
    (2024, 21, 'Autódromo José Carlos Pace'): 'São Paulo Grand Prix',
    (2024, 22, 'Las Vegas Strip Street Circuit'): 'Las Vegas Grand Prix',
    (2024, 23, 'Losail International Circuit'): 'Qatar Grand Prix',
    (2024, 24, 'Yas Marina Circuit'): 'Abu Dhabi Grand Prix',
}

# Define the fix function
def fix_race_name(row):
    # If the race_name is already correct, do nothing
    if pd.notna(row['race_name']):
        return row['race_name']

    # Create the lookup key from the row's data
    lookup_key = (row['season'], row['round'], row['circuit'])

    return manual_race_lookup.get(lookup_key, row['race_name'])


print("Checking for missing race names...")
nans_before = df['race_name'].isna().sum()
print(f"Found {nans_before} rows with missing race names.")

if nans_before > 0:
    print("Applying fixes...")
    df['race_name'] = df.apply(fix_race_name, axis=1)

    nans_after = df['race_name'].isna().sum()
    if nans_after == 0:
        print(f" Success! All {nans_before} missing names have been filled.")
    else:
        print(f" Incomplete. {nans_after} missing names still remain.")
else:
    print("No missing race names to fix.")

Checking for missing race names...
Found 373 rows with missing race names.
Applying fixes...
 Success! All 373 missing names have been filled.


In [21]:
race_details_map = {
    # 2018 Season
    (2018, 1, 'Albert Park Grand Prix Circuit'): {'date': '25-03-2018', 'time_of_race': '05:10:00', 'location': 'Melbourne', 'country': 'Australia'},
    (2018, 2, 'Bahrain International Circuit'): {'date': '08-04-2018', 'time_of_race': '15:10:00', 'location': 'Sakhir', 'country': 'Bahrain'},
    (2018, 3, 'Shanghai International Circuit'): {'date': '15-04-2018', 'time_of_race': '06:10:00', 'location': 'Shanghai', 'country': 'China'},
    (2018, 4, 'Baku City Circuit'): {'date': '29-04-2018', 'time_of_race': '12:10:00', 'location': 'Baku', 'country': 'Azerbaijan'},
    (2018, 5, 'Circuit de Barcelona-Catalunya'): {'date': '13-05-2018', 'time_of_race': '13:10:00', 'location': 'Montmeló', 'country': 'Spain'},
    (2018, 6, 'Circuit de Monaco'): {'date': '27-05-2018', 'time_of_race': '13:10:00', 'location': 'Monaco', 'country': 'Monaco'},
    (2018, 7, 'Circuit Gilles Villeneuve'): {'date': '10-06-2018', 'time_of_race': '18:10:00', 'location': 'Montréal', 'country': 'Canada'},
    (2018, 8, 'Circuit Paul Ricard'): {'date': '24-06-2018', 'time_of_race': '14:10:00', 'location': 'Le Castellet', 'country': 'France'},
    (2018, 9, 'Red Bull Ring'): {'date': '01-07-2018', 'time_of_race': '13:10:00', 'location': 'Spielberg', 'country': 'Austria'},
    (2018, 10, 'Silverstone Circuit'): {'date': '08-07-2018', 'time_of_race': '13:10:00', 'location': 'Silverstone', 'country': 'UK'},
    (2018, 11, 'Hockenheimring'): {'date': '22-07-2018', 'time_of_race': '13:10:00', 'location': 'Hockenheim', 'country': 'Germany'},
    (2018, 12, 'Hungaroring'): {'date': '29-07-2018', 'time_of_race': '13:10:00', 'location': 'Mogyoród', 'country': 'Hungary'},
    (2018, 13, 'Circuit de Spa-Francorchamps'): {'date': '26-08-2018', 'time_of_race': '13:10:00', 'location': 'Stavelot', 'country': 'Belgium'},
    (2018, 14, 'Autodromo Nazionale di Monza'): {'date': '02-09-2018', 'time_of_race': '13:10:00', 'location': 'Monza', 'country': 'Italy'},
    (2018, 15, 'Marina Bay Street Circuit'): {'date': '16-09-2018', 'time_of_race': '12:10:00', 'location': 'Singapore', 'country': 'Singapore'},
    (2018, 16, 'Sochi Autodrom'): {'date': '30-09-2018', 'time_of_race': '11:10:00', 'location': 'Sochi', 'country': 'Russia'},
    (2018, 17, 'Suzuka Circuit'): {'date': '07-10-2018', 'time_of_race': '05:10:00', 'location': 'Suzuka', 'country': 'Japan'},
    (2018, 18, 'Circuit of the Americas'): {'date': '21-10-2018', 'time_of_race': '18:10:00', 'location': 'Austin', 'country': 'USA'},
    (2018, 19, 'Autódromo Hermanos Rodríguez'): {'date': '28-10-2018', 'time_of_race': '19:10:00', 'location': 'Mexico City', 'country': 'Mexico'},
    (2018, 21, 'Yas Marina Circuit'): {'date': '25-11-2018', 'time_of_race': '13:10:00', 'location': 'Abu Dhabi', 'country': 'UAE'},

    # 2019 Season
    (2019, 1, 'Albert Park Grand Prix Circuit'): {'date': '17-03-2019', 'time_of_race': '05:10:00', 'location': 'Melbourne', 'country': 'Australia'},
    (2019, 2, 'Bahrain International Circuit'): {'date': '31-03-2019', 'time_of_race': '15:10:00', 'location': 'Sakhir', 'country': 'Bahrain'},
    (2019, 3, 'Shanghai International Circuit'): {'date': '14-04-2019', 'time_of_race': '06:10:00', 'location': 'Shanghai', 'country': 'China'},
    (2019, 4, 'Baku City Circuit'): {'date': '28-04-2019', 'time_of_race': '12:10:00', 'location': 'Baku', 'country': 'Azerbaijan'},
    (2019, 5, 'Circuit de Barcelona-Catalunya'): {'date': '12-05-2019', 'time_of_race': '13:10:00', 'location': 'Montmeló', 'country': 'Spain'},
    (2019, 6, 'Circuit de Monaco'): {'date': '26-05-2019', 'time_of_race': '13:10:00', 'location': 'Monaco', 'country': 'Monaco'},
    (2019, 7, 'Circuit Gilles Villeneuve'): {'date': '09-06-2019', 'time_of_race': '18:10:00', 'location': 'Montréal', 'country': 'Canada'},
    (2019, 8, 'Circuit Paul Ricard'): {'date': '23-06-2019', 'time_of_race': '13:10:00', 'location': 'Le Castellet', 'country': 'France'},
    (2019, 9, 'Red Bull Ring'): {'date': '30-06-2019', 'time_of_race': '13:10:00', 'location': 'Spielberg', 'country': 'Austria'},
    (2019, 10, 'Silverstone Circuit'): {'date': '14-07-2019', 'time_of_race': '13:10:00', 'location': 'Silverstone', 'country': 'UK'},
    (2019, 11, 'Hockenheimring'): {'date': '28-07-2019', 'time_of_race': '13:10:00', 'location': 'Hockenheim', 'country': 'Germany'},
    (2019, 12, 'Hungaroring'): {'date': '04-08-2019', 'time_of_race': '13:10:00', 'location': 'Mogyoród', 'country': 'Hungary'},
    (2019, 13, 'Circuit de Spa-Francorchamps'): {'date': '01-09-2019', 'time_of_race': '13:10:00', 'location': 'Stavelot', 'country': 'Belgium'},
    (2019, 14, 'Autodromo Nazionale di Monza'): {'date': '08-09-2019', 'time_of_race': '13:10:00', 'location': 'Monza', 'country': 'Italy'},
    (2019, 15, 'Marina Bay Street Circuit'): {'date': '22-09-2019', 'time_of_race': '12:10:00', 'location': 'Singapore', 'country': 'Singapore'},
    (2019, 16, 'Sochi Autodrom'): {'date': '29-09-2019', 'time_of_race': '11:10:00', 'location': 'Sochi', 'country': 'Russia'},
    (2019, 17, 'Suzuka Circuit'): {'date': '13-10-2019', 'time_of_race': '05:10:00', 'location': 'Suzuka', 'country': 'Japan'},
    (2019, 18, 'Autódromo Hermanos Rodríguez'): {'date': '27-10-2019', 'time_of_race': '19:10:00', 'location': 'Mexico City', 'country': 'Mexico'},
    (2019, 19, 'Circuit of the Americas'): {'date': '03-11-2019', 'time_of_race': '19:10:00', 'location': 'Austin', 'country': 'USA'},
    (2019, 21, 'Yas Marina Circuit'): {'date': '01-12-2019', 'time_of_race': '13:10:00', 'location': 'Abu Dhabi', 'country': 'UAE'},

    # 2020 Season
    (2020, 1, 'Red Bull Ring'): {'date': '05-07-2020', 'time_of_race': '13:10:00', 'location': 'Spielberg', 'country': 'Austria'},
    (2020, 2, 'Red Bull Ring'): {'date': '12-07-2020', 'time_of_race': '13:10:00', 'location': 'Spielberg', 'country': 'Austria'},
    (2020, 3, 'Hungaroring'): {'date': '19-07-2020', 'time_of_race': '13:10:00', 'location': 'Mogyoród', 'country': 'Hungary'},
    (2020, 4, 'Silverstone Circuit'): {'date': '02-08-2020', 'time_of_race': '13:10:00', 'location': 'Silverstone', 'country': 'UK'},
    (2020, 5, 'Silverstone Circuit'): {'date': '09-08-2020', 'time_of_race': '13:10:00', 'location': 'Silverstone', 'country': 'UK'},
    (2020, 6, 'Circuit de Barcelona-Catalunya'): {'date': '16-08-2020', 'time_of_race': '13:10:00', 'location': 'Montmeló', 'country': 'Spain'},
    (2020, 7, 'Circuit de Spa-Francorchamps'): {'date': '30-08-2020', 'time_of_race': '13:10:00', 'location': 'Stavelot', 'country': 'Belgium'},
    (2020, 8, 'Autodromo Nazionale di Monza'): {'date': '06-09-2020', 'time_of_race': '13:10:00', 'location': 'Monza', 'country': 'Italy'},
    (2020, 9, 'Autodromo Internazionale del Mugello'): {'date': '13-09-2020', 'time_of_race': '13:10:00', 'location': 'Mugello', 'country': 'Italy'},
    (2020, 10, 'Sochi Autodrom'): {'date': '27-09-2020', 'time_of_race': '11:10:00', 'location': 'Sochi', 'country': 'Russia'},
    (2020, 11, 'Nürburgring'): {'date': '11-10-2020', 'time_of_race': '12:10:00', 'location': 'Nürburg', 'country': 'Germany'},
    (2020, 12, 'Autódromo Internacional do Algarve'): {'date': '25-10-2020', 'time_of_race': '13:10:00', 'location': 'Portimão', 'country': 'Portugal'},
    (2020, 13, 'Autodromo Enzo e Dino Ferrari'): {'date': '01-11-2020', 'time_of_race': '12:10:00', 'location': 'Imola', 'country': 'Italy'},
    (2020, 14, 'Istanbul Park'): {'date': '15-11-2020', 'time_of_race': '10:10:00', 'location': 'Istanbul', 'country': 'Turkey'},
    (2020, 15, 'Bahrain International Circuit'): {'date': '29-11-2020', 'time_of_race': '14:10:00', 'location': 'Sakhir', 'country': 'Bahrain'},
    (2020, 16, 'Bahrain International Circuit'): {'date': '06-12-2020', 'time_of_race': '17:10:00', 'location': 'Sakhir', 'country': 'Bahrain'},
    (2020, 17, 'Yas Marina Circuit'): {'date': '13-12-2020', 'time_of_race': '13:10:00', 'location': 'Abu Dhabi', 'country': 'UAE'},

    # 2021 Season
    (2021, 1, 'Bahrain International Circuit'): {'date': '28-03-2021', 'time_of_race': '15:00:00', 'location': 'Sakhir', 'country': 'Bahrain'},
    (2021, 2, 'Autodromo Enzo e Dino Ferrari'): {'date': '18-04-2021', 'time_of_race': '13:00:00', 'location': 'Imola', 'country': 'Italy'},
    (2021, 3, 'Autódromo Internacional do Algarve'): {'date': '02-05-2021', 'time_of_race': '14:00:00', 'location': 'Portimão', 'country': 'Portugal'},
    (2021, 4, 'Circuit de Barcelona-Catalunya'): {'date': '09-05-2021', 'time_of_race': '13:00:00', 'location': 'Montmeló', 'country': 'Spain'},
    (2021, 5, 'Circuit de Monaco'): {'date': '23-05-2021', 'time_of_race': '13:00:00', 'location': 'Monaco', 'country': 'Monaco'},
    (2021, 6, 'Baku City Circuit'): {'date': '06-06-2021', 'time_of_race': '12:00:00', 'location': 'Baku', 'country': 'Azerbaijan'},
    (2021, 7, 'Circuit Paul Ricard'): {'date': '20-06-2021', 'time_of_race': '13:00:00', 'location': 'Le Castellet', 'country': 'France'},
    (2021, 8, 'Red Bull Ring'): {'date': '27-06-2021', 'time_of_race': '13:00:00', 'location': 'Spielberg', 'country': 'Austria'},
    (2021, 9, 'Red Bull Ring'): {'date': '04-07-2021', 'time_of_race': '13:00:00', 'location': 'Spielberg', 'country': 'Austria'},
    (2021, 10, 'Silverstone Circuit'): {'date': '18-07-2021', 'time_of_race': '14:00:00', 'location': 'Silverstone', 'country': 'UK'},
    (2021, 11, 'Hungaroring'): {'date': '01-08-2021', 'time_of_race': '13:00:00', 'location': 'Mogyoród', 'country': 'Hungary'},
    (2021, 12, 'Circuit de Spa-Francorchamps'): {'date': '29-08-2021', 'time_of_race': '13:00:00', 'location': 'Stavelot', 'country': 'Belgium'},
    (2021, 13, 'Circuit Park Zandvoort'): {'date': '05-09-2021', 'time_of_race': '13:00:00', 'location': 'Zandvoort', 'country': 'Netherlands'},
    (2021, 14, 'Autodromo Nazionale di Monza'): {'date': '12-09-2021', 'time_of_race': '13:00:00', 'location': 'Monza', 'country': 'Italy'},
    (2021, 15, 'Sochi Autodrom'): {'date': '26-09-2021', 'time_of_race': '12:00:00', 'location': 'Sochi', 'country': 'Russia'},
    (2021, 16, 'Istanbul Park'): {'date': '10-10-2021', 'time_of_race': '12:00:00', 'location': 'Istanbul', 'country': 'Turkey'},
    (2021, 17, 'Circuit of the Americas'): {'date': '24-10-2021', 'time_of_race': '19:00:00', 'location': 'Austin', 'country': 'USA'},
    (2021, 20, 'Losail International Circuit'): {'date': '21-11-2021', 'time_of_race': '14:00:00', 'location': 'Lusail', 'country': 'Qatar'},
    (2021, 21, 'Jeddah Corniche Circuit'): {'date': '05-12-2021', 'time_of_race': '17:30:00', 'location': 'Jeddah', 'country': 'Saudi Arabia'},
    (2021, 22, 'Yas Marina Circuit'): {'date': '12-12-2021', 'time_of_race': '13:00:00', 'location': 'Abu Dhabi', 'country': 'UAE'},

    # 2022 Season
    (2022, 1, 'Bahrain International Circuit'): {'date': '20-03-2022', 'time_of_race': '15:00:00', 'location': 'Sakhir', 'country': 'Bahrain'},
    (2022, 2, 'Jeddah Corniche Circuit'): {'date': '27-03-2022', 'time_of_race': '17:00:00', 'location': 'Jeddah', 'country': 'Saudi Arabia'},
    (2022, 3, 'Albert Park Grand Prix Circuit'): {'date': '10-04-2022', 'time_of_race': '05:00:00', 'location': 'Melbourne', 'country': 'Australia'},
    (2022, 4, 'Autodromo Enzo e Dino Ferrari'): {'date': '24-04-2022', 'time_of_race': '13:00:00', 'location': 'Imola', 'country': 'Italy'},
    (2022, 5, 'Miami International Autodrome'): {'date': '08-05-2022', 'time_of_race': '19:30:00', 'location': 'Miami', 'country': 'USA'},
    (2022, 6, 'Circuit de Barcelona-Catalunya'): {'date': '22-05-2022', 'time_of_race': '13:00:00', 'location': 'Montmeló', 'country': 'Spain'},
    (2022, 7, 'Circuit de Monaco'): {'date': '29-05-2022', 'time_of_race': '13:00:00', 'location': 'Monaco', 'country': 'Monaco'},
    (2022, 8, 'Baku City Circuit'): {'date': '12-06-2022', 'time_of_race': '11:00:00', 'location': 'Baku', 'country': 'Azerbaijan'},
    (2022, 9, 'Circuit Gilles Villeneuve'): {'date': '19-06-2022', 'time_of_race': '18:00:00', 'location': 'Montréal', 'country': 'Canada'},
    (2022, 10, 'Silverstone Circuit'): {'date': '03-07-2022', 'time_of_race': '14:00:00', 'location': 'Silverstone', 'country': 'UK'},
    (2022, 11, 'Red Bull Ring'): {'date': '10-07-2022', 'time_of_race': '13:00:00', 'location': 'Spielberg', 'country': 'Austria'},
    (2022, 12, 'Circuit Paul Ricard'): {'date': '24-07-2022', 'time_of_race': '13:00:00', 'location': 'Le Castellet', 'country': 'France'},
    (2022, 13, 'Hungaroring'): {'date': '31-07-2022', 'time_of_race': '13:00:00', 'location': 'Mogyoród', 'country': 'Hungary'},
    (2022, 14, 'Circuit de Spa-Francorchamps'): {'date': '28-08-2022', 'time_of_race': '13:00:00', 'location': 'Stavelot', 'country': 'Belgium'},
    (2022, 15, 'Circuit Park Zandvoort'): {'date': '04-09-2022', 'time_of_race': '13:00:00', 'location': 'Zandvoort', 'country': 'Netherlands'},
    (2022, 16, 'Autodromo Nazionale di Monza'): {'date': '11-09-2022', 'time_of_race': '13:00:00', 'location': 'Monza', 'country': 'Italy'},
    (2022, 17, 'Marina Bay Street Circuit'): {'date': '02-10-2022', 'time_of_race': '12:00:00', 'location': 'Singapore', 'country': 'Singapore'},
    (2022, 18, 'Suzuka Circuit'): {'date': '09-10-2022', 'time_of_race': '05:00:00', 'location': 'Suzuka', 'country': 'Japan'},
    (2022, 19, 'Circuit of the Americas'): {'date': '23-10-2022', 'time_of_race': '19:00:00', 'location': 'Austin', 'country': 'USA'},
    (2022, 20, 'Autódromo Hermanos Rodríguez'): {'date': '30-10-2022', 'time_of_race': '20:00:00', 'location': 'Mexico City', 'country': 'Mexico'},
    (2022, 22, 'Yas Marina Circuit'): {'date': '20-11-2022', 'time_of_race': '13:00:00', 'location': 'Abu Dhabi', 'country': 'UAE'},

    # 2023 Season
    (2023, 1, 'Bahrain International Circuit'): {'date': '05-03-2023', 'time_of_race': '15:00:00', 'location': 'Sakhir', 'country': 'Bahrain'},
    (2023, 2, 'Jeddah Corniche Circuit'): {'date': '19-03-2023', 'time_of_race': '17:00:00', 'location': 'Jeddah', 'country': 'Saudi Arabia'},
    (2023, 3, 'Albert Park Grand Prix Circuit'): {'date': '02-04-2023', 'time_of_race': '05:00:00', 'location': 'Melbourne', 'country': 'Australia'},
    (2023, 4, 'Baku City Circuit'): {'date': '30-04-2023', 'time_of_race': '11:00:00', 'location': 'Baku', 'country': 'Azerbaijan'},
    (2023, 5, 'Miami International Autodrome'): {'date': '07-05-2023', 'time_of_race': '19:30:00', 'location': 'Miami', 'country': 'USA'},
    (2023, 6, 'Circuit de Monaco'): {'date': '28-05-2023', 'time_of_race': '13:00:00', 'location': 'Monaco', 'country': 'Monaco'},
    (2023, 7, 'Circuit de Barcelona-Catalunya'): {'date': '04-06-2023', 'time_of_race': '13:00:00', 'location': 'Montmeló', 'country': 'Spain'},
    (2023, 8, 'Circuit Gilles Villeneuve'): {'date': '18-06-2023', 'time_of_race': '18:00:00', 'location': 'Montréal', 'country': 'Canada'},
    (2023, 9, 'Red Bull Ring'): {'date': '02-07-2023', 'time_of_race': '13:00:00', 'location': 'Spielberg', 'country': 'Austria'},
    (2023, 10, 'Silverstone Circuit'): {'date': '09-07-2023', 'time_of_race': '14:00:00', 'location': 'Silverstone', 'country': 'UK'},
    (2023, 11, 'Hungaroring'): {'date': '23-07-2023', 'time_of_race': '13:00:00', 'location': 'Mogyoród', 'country': 'Hungary'},
    (2023, 12, 'Circuit de Spa-Francorchamps'): {'date': '30-07-2023', 'time_of_race': '13:00:00', 'location': 'Stavelot', 'country': 'Belgium'},
    (2023, 13, 'Circuit Park Zandvoort'): {'date': '27-08-2023', 'time_of_race': '13:00:00', 'location': 'Zandvoort', 'country': 'Netherlands'},
    (2023, 14, 'Autodromo Nazionale di Monza'): {'date': '03-09-2023', 'time_of_race': '13:00:00', 'location': 'Monza', 'country': 'Italy'},
    (2023, 15, 'Marina Bay Street Circuit'): {'date': '17-09-2023', 'time_of_race': '12:00:00', 'location': 'Singapore', 'country': 'Singapore'},
    (2023, 16, 'Suzuka Circuit'): {'date': '24-09-2023', 'time_of_race': '05:00:00', 'location': 'Suzuka', 'country': 'Japan'},
    (2023, 17, 'Losail International Circuit'): {'date': '08-10-2023', 'time_of_race': '17:00:00', 'location': 'Lusail', 'country': 'Qatar'},
    (2023, 18, 'Circuit of the Americas'): {'date': '22-10-2023', 'time_of_race': '19:00:00', 'location': 'Austin', 'country': 'USA'},
    (2023, 19, 'Autódromo Hermanos Rodríguez'): {'date': '29-10-2023', 'time_of_race': '20:00:00', 'location': 'Mexico City', 'country': 'Mexico'},
    (2023, 21, 'Las Vegas Strip Street Circuit'): {'date': '19-11-2023', 'time_of_race': '06:00:00', 'location': 'Las Vegas', 'country': 'USA'},
    (2023, 22, 'Yas Marina Circuit'): {'date': '26-11-2023', 'time_of_race': '13:00:00', 'location': 'Abu Dhabi', 'country': 'UAE'},

    # 2024 Season
    (2024, 1, 'Bahrain International Circuit'): {'date': '02-03-2024', 'time_of_race': '15:00:00', 'location': 'Sakhir', 'country': 'Bahrain'},
    (2024, 2, 'Jeddah Corniche Circuit'): {'date': '09-03-2024', 'time_of_race': '17:00:00', 'location': 'Jeddah', 'country': 'Saudi Arabia'},
    (2024, 3, 'Albert Park Grand Prix Circuit'): {'date': '24-03-2024', 'time_of_race': '04:00:00', 'location': 'Melbourne', 'country': 'Australia'},
    (2024, 4, 'Suzuka Circuit'): {'date': '07-04-2024', 'time_of_race': '05:00:00', 'location': 'Suzuka', 'country': 'Japan'},
    (2024, 5, 'Shanghai International Circuit'): {'date': '21-04-2024', 'time_of_race': '07:00:00', 'location': 'Shanghai', 'country': 'China'},
    (2024, 6, 'Miami International Autodrome'): {'date': '05-05-2024', 'time_of_race': '20:00:00', 'location': 'Miami', 'country': 'USA'},
    (2024, 7, 'Autodromo Enzo e Dino Ferrari'): {'date': '19-05-2024', 'time_of_race': '13:00:00', 'location': 'Imola', 'country': 'Italy'},
    (2024, 8, 'Circuit de Monaco'): {'date': '26-05-2024', 'time_of_race': '13:00:00', 'location': 'Monaco', 'country': 'Monaco'},
    (2024, 9, 'Circuit Gilles Villeneuve'): {'date': '09-06-2024', 'time_of_race': '18:00:00', 'location': 'Montréal', 'country': 'Canada'},
    (2024, 10, 'Circuit de Barcelona-Catalunya'): {'date': '23-06-2024', 'time_of_race': '13:00:00', 'location': 'Montmeló', 'country': 'Spain'},
    (2024, 11, 'Red Bull Ring'): {'date': '30-06-2024', 'time_of_race': '13:00:00', 'location': 'Spielberg', 'country': 'Austria'},
    (2024, 12, 'Silverstone Circuit'): {'date': '07-07-2024', 'time_of_race': '14:00:00', 'location': 'Silverstone', 'country': 'UK'},
    (2024, 13, 'Hungaroring'): {'date': '21-07-2024', 'time_of_race': '13:00:00', 'location': 'Mogyoród', 'country': 'Hungary'},
    (2024, 14, 'Circuit de Spa-Francorchamps'): {'date': '28-07-2024', 'time_of_race': '13:00:00', 'location': 'Stavelot', 'country': 'Belgium'},
    (2024, 15, 'Circuit Park Zandvoort'): {'date': '25-08-2024', 'time_of_race': '13:00:00', 'location': 'Zandvoort', 'country': 'Netherlands'},
    (2024, 16, 'Autodromo Nazionale di Monza'): {'date': '01-09-2024', 'time_of_race': '13:00:00', 'location': 'Monza', 'country': 'Italy'},
    (2024, 17, 'Baku City Circuit'): {'date': '15-09-2024', 'time_of_race': '11:00:00', 'location': 'Baku', 'country': 'Azerbaijan'},
    (2024, 18, 'Marina Bay Street Circuit'): {'date': '22-09-2024', 'time_of_race': '12:00:00', 'location': 'Singapore', 'country': 'Singapore'},
    (2024, 19, 'Circuit of the Americas'): {'date': '20-10-2024', 'time_of_race': '19:00:00', 'location': 'Austin', 'country': 'USA'},
    (2024, 20, 'Autódromo Hermanos Rodríguez'): {'date': '27-10-2024', 'time_of_race': '20:00:00', 'location': 'Mexico City', 'country': 'Mexico'},
    (2024, 21, 'Autódromo José Carlos Pace'): {'date': '03-11-2024', 'time_of_race': '17:00:00', 'location': 'São Paulo', 'country': 'Brazil'},
    (2024, 22, 'Las Vegas Strip Street Circuit'): {'date': '24-11-2024', 'time_of_race': '06:00:00', 'location': 'Las Vegas', 'country': 'USA'},
    (2024, 23, 'Losail International Circuit'): {'date': '01-12-2024', 'time_of_race': '17:00:00', 'location': 'Lusail', 'country': 'Qatar'},
    (2024, 24, 'Yas Marina Circuit'): {'date': '08-12-2024', 'time_of_race': '13:00:00', 'location': 'Abu Dhabi', 'country': 'UAE'},
}


def fill_race_details(row):
    lookup_key = (row['season'], row['round'], row['circuit'])

    if lookup_key in race_details_map:
        correct_details = race_details_map[lookup_key]

        # Fill 'date' if it's missing
        if pd.isna(row['date']):
            row['date'] = correct_details['date']

        # Fill 'time_of_race' if it's missing
        if pd.isna(row['time_of_race']):
            row['time_of_race'] = correct_details['time_of_race']

        # Fill 'location' if it's missing
        if pd.isna(row['location']):
            row['location'] = correct_details['location']

        # Fill 'country' if it's missing (NEW)
        if pd.isna(row['country']):
            row['country'] = correct_details['country']

    return row

print("Checking for missing race details...")
columns_to_check = ['date', 'time_of_race', 'location', 'country']
nans_before = df[columns_to_check].isna().sum()
total_nans_before = nans_before.sum()
print(f"Found {total_nans_before} total missing values across the four columns.")

if total_nans_before > 0:
    print("Applying fixes...")
    df = df.apply(fill_race_details, axis=1)

    nans_after = df[columns_to_check].isna().sum()
    total_nans_after = nans_after.sum()

    if total_nans_after == 0:
        print(f"Success! All {total_nans_before} missing details have been filled.")
    else:
        print(f"Incomplete. {total_nans_after} missing details still remain.")
        print("Remaining NaNs per column:\n", nans_after[nans_after > 0])
else:
    print("No missing race details to fix.")

Checking for missing race details...
Found 1492 total missing values across the four columns.
Applying fixes...
Success! All 1492 missing details have been filled.


In [25]:
weather_details_map = {
    # 2018
    (2018, 19, 'Autódromo Hermanos Rodríguez'): {'air_temp_c': 21.5, 'track_temp_c': 39.8, 'humidity_%': 35.2, 'wind_speed_kmh': 4.5},
    (2018, 20, 'Autódromo José Carlos Pace'): {'air_temp_c': 23.1, 'track_temp_c': 38.5, 'humidity_%': 70.1, 'wind_speed_kmh': 7.2},

    # 2019
    (2019, 18, 'Autódromo Hermanos Rodríguez'): {'air_temp_c': 22.3, 'track_temp_c': 47.1, 'humidity_%': 24.5, 'wind_speed_kmh': 5.8},
    (2019, 20, 'Autódromo José Carlos Pace'): {'air_temp_c': 20.4, 'track_temp_c': 30.2, 'humidity_%': 75.0, 'wind_speed_kmh': 11.2},

    # 2020
    (2020, 11, 'Nürburgring'): {'air_temp_c': 8.5, 'track_temp_c': 16.5, 'humidity_%': 87.0, 'wind_speed_kmh': 9.5},
    (2020, 12, 'Autódromo Internacional do Algarve'): {'air_temp_c': 19.8, 'track_temp_c': 29.5, 'humidity_%': 65.0, 'wind_speed_kmh': 18.5},

    # 2021
    (2021, 18, 'Autódromo Hermanos Rodríguez'): {'air_temp_c': 20.8, 'track_temp_c': 48.2, 'humidity_%': 22.0, 'wind_speed_kmh': 6.0},
    (2021, 19, 'Autódromo José Carlos Pace'): {'air_temp_c': 18.5, 'track_temp_c': 35.1, 'humidity_%': 82.0, 'wind_speed_kmh': 14.0},

    # 2022
    (2022, 20, 'Autódromo Hermanos Rodríguez'): {'air_temp_c': 23.5, 'track_temp_c': 48.9, 'humidity_%': 18.0, 'wind_speed_kmh': 7.5},
    (2022, 21, 'Autódromo José Carlos Pace'): {'air_temp_c': 22.0, 'track_temp_c': 50.3, 'humidity_%': 60.0, 'wind_speed_kmh': 10.1},

    # 2023
    (2023, 19, 'Autódromo Hermanos Rodríguez'): {'air_temp_c': 24.1, 'track_temp_c': 49.5, 'humidity_%': 21.0, 'wind_speed_kmh': 5.5},
    (2023, 20, 'Autódromo José Carlos Pace'): {'air_temp_c': 21.7, 'track_temp_c': 45.6, 'humidity_%': 72.0, 'wind_speed_kmh': 8.3},
}

#  Define the function to apply the manual fix
def fill_weather_details(row):
    lookup_key = (row['season'], row['round'], row['circuit'])
    if lookup_key in weather_details_map:
        correct_details = weather_details_map[lookup_key]
        if pd.isna(row['air_temp_c']): row['air_temp_c'] = correct_details['air_temp_c']
        if pd.isna(row['track_temp_c']): row['track_temp_c'] = correct_details['track_temp_c']
        if pd.isna(row['humidity_%']): row['humidity_%'] = correct_details['humidity_%']
        if pd.isna(row['wind_speed_kmh']): row['wind_speed_kmh'] = correct_details['wind_speed_kmh']
    return row

# Apply the multi-layered imputation
weather_columns = ['air_temp_c', 'track_temp_c', 'humidity_%', 'wind_speed_kmh']
print("Starting multi-layered imputation...")

# Layer 1: Apply manual fixes
print("Applying manual fixes...")
df = df.apply(fill_weather_details, axis=1)

# Layer 2: Apply country-level average for remaining NaNs
print("Applying country-level average as fallback...")
for col in weather_columns:
    df[col] = df.groupby('country')[col].transform(lambda x: x.fillna(x.mean()))

# Layer 3: Apply global average for any final, stubborn NaNs (FutureWarning-safe)
print("Applying global average as final guarantee...")
for col in weather_columns:
    global_mean = df[col].mean()
    df[col] = df[col].fillna(global_mean)

print("\nImputation complete.")

final_nans = df[weather_columns].isna().sum().sum()
if final_nans == 0:
    print(" Success! All missing weather data has been filled.")
else:
    print(f" Error. {final_nans} NaNs still remain after all steps.")

Starting multi-layered imputation...
Applying manual fixes...
Applying country-level average as fallback...
Applying global average as final guarantee...

Imputation complete.
 Success! All missing weather data has been filled.


In [26]:
# Find all rows where 'lap_time_variation' is NaN
missing_variation_df = df[df['lap_time_variation'].isna()]

# Check the distribution of the 'laps' column for those specific rows
lap_counts_for_missing_data = missing_variation_df['laps'].value_counts().sort_index()

print("Distribution of laps completed for rows with missing lap_time_variation:")
print(lap_counts_for_missing_data)

Distribution of laps completed for rows with missing lap_time_variation:
laps
0     66
1     35
2      2
3      4
4      2
5      7
6     10
7      4
8      6
9      6
10     3
11     3
13     3
14     3
15     1
17     2
18     2
19     1
20     2
22     2
23     1
24     2
25     3
26     2
27     2
28     1
29     2
31     2
38     1
48     2
53     1
54     1
57     1
Name: count, dtype: int64


In [27]:
missing_aggression_df = df[df['tire_usage_aggression'].isna()]

# Check the value counts of the 'laps' column for these rows
lap_counts = missing_aggression_df['laps'].value_counts()

print("Distribution of laps completed for rows with missing tire_usage_aggression:")
print(lap_counts)

Distribution of laps completed for rows with missing tire_usage_aggression:
laps
0    66
Name: count, dtype: int64


In [28]:
missing_attempts_df = df[df['fast_lap_attempts'].isna()]

# Check the distribution of the 'laps' column for those rows
lap_counts_for_missing_data = missing_attempts_df['laps'].value_counts().sort_index()

print("Distribution of laps completed for rows with missing fast_lap_attempts:")
print(lap_counts_for_missing_data)

Distribution of laps completed for rows with missing fast_lap_attempts:
laps
0     66
1     35
2      2
3      4
4      2
5      7
6     10
7      4
8      6
9      6
10     3
11     3
13     3
14     3
15     1
17     2
18     2
19     1
20     2
22     2
23     1
24     2
25     3
26     2
27     2
28     1
29     2
31     2
38     1
48     2
53     1
54     1
57     1
Name: count, dtype: int64


In [29]:
missing_score_df = df[df['driver_aggression_score'].isna()]

lap_counts_for_missing_score = missing_score_df['laps'].value_counts().sort_index()

print("Distribution of laps completed for rows with missing driver_aggression_score:")
print(lap_counts_for_missing_score)

Distribution of laps completed for rows with missing driver_aggression_score:
laps
0     66
1     35
2      2
3      4
4      2
5      7
6     10
7      4
8      6
9      6
10     3
11     3
13     3
14     3
15     1
17     2
18     2
19     1
20     2
22     2
23     1
24     2
25     3
26     2
27     2
28     1
29     2
31     2
38     1
48     2
53     1
54     1
57     1
Name: count, dtype: int64


In [ ]:
output_filename = 'f1_data_cleaned.xlsx'

df.to_excel(output_filename, index=False)

print(f"✅ Your cleaned data has been successfully saved as: {output_filename}")

✅ Your cleaned data has been successfully saved as: f1_data_cleaned.xlsx


# **Dealing with the stint logic**



In [ ]:
file_path = r'f1_data_cleaned_P1.xlsx'

try:
    df2 = pd.read_excel(file_path)

    print("File loaded successfully into df2.")
    display(df2.head())

except FileNotFoundError:
    print(f"ERROR: The file was not found at the path: '{file_path}'")
    print("Please check the 'What to Do If You Get an Error' section below.")

File loaded successfully into df2.


,season,round,circuit,driver,constructor,laps,position,totalpitstops,avgpitstoptime,race_name,...,abbreviation,stint,tire_compound,stint_length,pit_lap,pit_time,tire_compound_code,year,month,day
0,2018,1,Albert Park Grand Prix Circuit,Sebastian Vettel,Ferrari,58,1,1,21.787,Australian Grand Prix,...,VET,1.0,ULTRASOFT,25.0,26.0,21.787,C4,2018,3,25
1,2018,1,Albert Park Grand Prix Circuit,Sebastian Vettel,Ferrari,58,1,1,21.787,Australian Grand Prix,...,VET,2.0,SOFT,32.0,NaN,-1.000,C5,2018,3,25
2,2018,1,Albert Park Grand Prix Circuit,Lewis Hamilton,Mercedes,58,2,1,21.821,Australian Grand Prix,...,HAM,1.0,ULTRASOFT,17.0,19.0,21.821,C4,2018,3,25
3,2018,1,Albert Park Grand Prix Circuit,Lewis Hamilton,Mercedes,58,2,1,21.821,Australian Grand Prix,...,HAM,2.0,SOFT,39.0,NaN,-1.000,C5,2018,3,25
4,2018,1,Albert Park Grand Prix Circuit,Kimi Raikkönen,Ferrari,58,3,1,21.421,Australian Grand Prix,...,RAI,1.0,ULTRASOFT,17.0,18.0,21.421,C4,2018,3,25


Step 1: Primary Validation - Checking Structural Integrity
This initial step is crucial for ensuring our data is reliable. Before we perform any calculations or corrections, we must verify that the dataset has a sound structure.

The code in the following cell groups the data for each unique driver in each race and performs several critical checks to find structural errors. Specifically, it verifies that for every group:

Rule 1: Correct Number of Rows

The total number of rows must exactly equal totalpitstops + 1. For example, a driver with 2 pit stops must have 3 rows of data.

Rule 2: Consistent pit_lap Count

The number of valid (non-empty) pit_lap entries must exactly equal totalpitstops.

Rule 3: Consistent pit_time Count

The number of actual pit stop times (where pit_time is not -1) must also exactly equal totalpitstops.

Any group that fails one or more of these checks is flagged as "inconsistent". This indicates fundamental issues like missing or extra rows, which must be handled before we can trust the data for further analysis.



Groups data by driver & race.

Checks:
- Do rows = total pitstops + 1?
- Do pit_lap counts = total pitstops?
- Do valid pit_time counts = total pitstops?

Flags inconsistent groups and prints a sample for debugging.

In [ ]:

# --- Define the columns that uniquely identify a driver's race ---
group_cols = ['season', 'round', 'driver']

print("Starting Step 1: Primary Validation...")

# --- Perform all checks using groupby().transform() for efficiency ---
try:
    # Get the size of each group (total rows per driver/race)
    group_size = df2.groupby(group_cols)[group_cols[0]].transform('size')

    # Get the count of valid pit_lap entries per group
    valid_pit_lap_count = df2.groupby(group_cols)['pit_lap'].transform('count')

    # Get the count of actual pit stops from pit_time (where it's not -1)
    is_actual_pitstop = (df2['pit_time'] != -1) & (df2['pit_time'].notna())
    actual_pit_time_count = is_actual_pitstop.groupby([df2[col] for col in group_cols]).transform('sum')

    # Get the 'totalpitstops' value for comparison across all rows of a group
    totalpitstops_per_row = df2.groupby(group_cols)['totalpitstops'].transform('first')


    # --- Create boolean flags for each validation check ---
    df2['flag_row_count_ok'] = (group_size == (totalpitstops_per_row + 1))
    df2['flag_pit_lap_count_ok'] = (valid_pit_lap_count == totalpitstops_per_row)
    df2['flag_pit_time_count_ok'] = (actual_pit_time_count == totalpitstops_per_row)

    # A group is consistent only if all flags are True
    df2['is_consistent'] = (df2['flag_row_count_ok'] &
                            df2['flag_pit_lap_count_ok'] &
                            df2['flag_pit_time_count_ok'])

    # --- Report the findings ---
    inconsistent_groups = df2[~df2['is_consistent']].copy()

    if inconsistent_groups.empty:
        print("\n Success! All driver/race groups passed the primary validation checks.")
    else:
        unique_inconsistent = inconsistent_groups.drop_duplicates(subset=group_cols)
        num_inconsistent_groups = len(unique_inconsistent)
        total_groups = len(df2.drop_duplicates(subset=group_cols))

        print(f"\n🚨 Validation Complete: Found issues in {num_inconsistent_groups} out of {total_groups} driver/race groups.")

        # Add details for debugging
        inconsistent_groups['expected_rows'] = inconsistent_groups['totalpitstops'] + 1
        inconsistent_groups['actual_rows'] = group_size
        inconsistent_groups['actual_valid_pitlaps'] = valid_pit_lap_count
        inconsistent_groups['actual_valid_pittimes'] = actual_pit_time_count

        display_cols = group_cols + [
            'totalpitstops', 'expected_rows', 'actual_rows',
            'actual_valid_pitlaps', 'actual_valid_pittimes',
            'stint', 'pit_lap', 'pit_time'
        ]

        print("\n--- Sample of Inconsistent Groups ---")
        display(inconsistent_groups[display_cols].head(10))

    print("\nStep 1 is complete.")

except KeyError as e:
    print(f"\n❌ ERROR: A required column was not found: {e}")
    print("Please make sure your DataFrame 'df2' has the columns: 'season', 'round', 'driver', 'totalpitstops', 'pit_lap', and 'pit_time'.")
except NameError:
    print("\n❌ ERROR: The DataFrame 'df2' is not defined.")
    print("Please make sure you have loaded your data into a DataFrame named 'df2' before running this script.")

Starting Step 1: Primary Validation...

🚨 Validation Complete: Found issues in 181 out of 2821 driver/race groups.

--- Sample of Inconsistent Groups ---


,season,round,driver,totalpitstops,expected_rows,actual_rows,actual_valid_pitlaps,actual_valid_pittimes,stint,pit_lap,pit_time
16,2018,1,Stoffel Vandoorne,1,2,1,0,0,NaN,NaN,NaN
29,2018,1,Brendon Hartley,2,3,2,1,1,2.0,22.0,22.296
30,2018,1,Brendon Hartley,2,3,2,1,1,3.0,NaN,-1.000
31,2018,1,Romain Grosjean,1,2,1,1,1,1.0,24.0,23.054
32,2018,1,Kevin Magnussen,1,2,1,1,1,1.0,22.0,21.983
54,2018,2,Stoffel Vandoorne,2,3,1,0,0,NaN,NaN,NaN
73,2018,2,Sergey Sirotkin,2,3,1,0,0,NaN,NaN,NaN
85,2018,3,Daniel Ricciardo,2,3,2,1,1,1.0,17.0,23.042
86,2018,3,Daniel Ricciardo,2,3,2,1,1,3.0,NaN,-1.000
114,2018,3,Stoffel Vandoorne,1,2,1,0,0,NaN,NaN,NaN



Step 1 is complete.


In [ ]:
print("--- Preparing for Step 2 ---")
try:
    # --- Part 1: Create a new, clean DataFrame ---
    # This is the crucial step. We filter df2 to keep only the rows belonging to consistent groups.
    df2_cleaned = df2[df2['is_consistent']].copy()
    print(f"Created a new clean DataFrame 'df2_cleaned' with {len(df2_cleaned)} rows.")

    # --- Part 2: Perform the Secondary Validation on the clean data ---
    print("\nStarting Step 2: Secondary Validation - Checking pit_lap order...")

    # Define a function to check if a series is strictly increasing
    # It handles the NaN value for the final stint correctly.
    def is_strictly_increasing(series):
        # Drop NaNs, get the difference between consecutive laps, and check if all differences are positive.
        return series.dropna().diff().iloc[1:].gt(0).all()

    # Group by driver/race and apply the check
    group_cols = ['season', 'round', 'driver']
    pit_lap_order_ok = df2_cleaned.groupby(group_cols)['pit_lap'].apply(is_strictly_increasing)

    # Find and report any groups that fail the check
    inconsistent_order_groups = pit_lap_order_ok[~pit_lap_order_ok]

    if inconsistent_order_groups.empty:
        print("\n✅ Success! All groups have a correct and strictly increasing pit_lap order.")
    else:
        print(f"\n🚨 Validation Complete: Found {len(inconsistent_order_groups)} groups with pit_lap order errors.")

        # Get the full data for the problematic groups to display for review
        inconsistent_df = df2_cleaned[df2_cleaned.set_index(group_cols).index.isin(inconsistent_order_groups.index)]

        print("\n--- Sample of Groups with Incorrect pit_lap Order ---")
        display(inconsistent_df)

    print("\nStep 2 is complete.")

except NameError:
    print("\n❌ ERROR: The DataFrame 'df2' is not defined.")
    print("Please make sure you have run the Step 1 script successfully in a cell above this one.")
except KeyError:
    print("\n❌ ERROR: The 'is_consistent' column was not found in df2.")
    print("This column is created by the Step 1 script. Please ensure Step 1 was run successfully.")

--- Preparing for Step 2 ---
Created a new clean DataFrame 'df2_cleaned' with 7084 rows.

Starting Step 2: Secondary Validation - Checking pit_lap order...

✅ Success! All groups have a correct and strictly increasing pit_lap order.

Step 2 is complete.


In [ ]:
# Assuming 'df2_cleaned' is your DataFrame from the previous step.

print("Starting Step 3: Correction and Regeneration...")

# --- Part 1: Regenerate the 'stint' column ---
# We use cumcount() within each group to create a perfect sequence starting from 0, so we add 1.
group_cols = ['season', 'round', 'driver']
df2_cleaned['stint_corrected'] = df2_cleaned.groupby(group_cols).cumcount() + 1
print("Successfully regenerated the 'stint' column into 'stint_corrected'.")


# --- Part 2: Recalculate the 'stint_length' column ---

# Define the function that applies our rules to each driver/race group
def calculate_stint_lengths(group):
    # Sort by stint to be safe
    group = group.sort_values(by='stint_corrected')

    # Get the total laps for this group (it's the same for all rows in the group)
    total_laps = group['laps'].iloc[0]

    # Handle the simple case of a 0-stop race
    if group['totalpitstops'].iloc[0] == 0:
        group['stint_length_corrected'] = total_laps
        return group

    # Calculate pit lap differences within the group
    # The first stint's length is its pit_lap. Subsequent stints are the difference.
    pit_laps = group['pit_lap'].shift(1, fill_value=0)
    group['stint_length_corrected'] = group['pit_lap'] - pit_laps

    # Correct the final stint's length
    # It's the total laps minus the lap of the last pit stop.
    last_pit_lap = group['pit_lap'].max()
    final_stint_length = total_laps - last_pit_lap

    # Replace the calculated value for the final row (which will be wrong due to NaN) with the correct one.
    group.loc[group['pit_lap'].isnull(), 'stint_length_corrected'] = final_stint_length

    return group

# Apply the function to every group.
# We need to group by the original index to merge the results back correctly.
df2_cleaned = df2_cleaned.groupby(group_cols, group_keys=False).apply(calculate_stint_lengths)

# Convert the new column to an integer type
df2_cleaned['stint_length_corrected'] = df2_cleaned['stint_length_corrected'].astype(int)

print("Successfully recalculated the 'stint_length' column into 'stint_length_corrected'.")

# --- Display a sample to show the old vs. new columns ---
print("\n--- Sample of changes (showing a driver with multiple pit stops) ---")
# Find a driver with more than one pit stop to show a good example
sample_driver = df2_cleaned[df2_cleaned['totalpitstops'] > 1].head(5)
display(sample_driver[['driver', 'stint', 'stint_corrected', 'pit_lap', 'laps', 'stint_length', 'stint_length_corrected']])

print("\nStep 3 is complete.")

Starting Step 3: Correction and Regeneration...
Successfully regenerated the 'stint' column into 'stint_corrected'.
Successfully recalculated the 'stint_length' column into 'stint_length_corrected'.

--- Sample of changes (showing a driver with multiple pit stops) ---


/tmp/ipython-input-2002522535.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df2_cleaned = df2_cleaned.groupby(group_cols, group_keys=False).apply(calculate_stint_lengths)


,driver,stint,stint_corrected,pit_lap,laps,stint_length,stint_length_corrected
23,Charles Leclerc,1.0,1,20.0,58,19.0,20
24,Charles Leclerc,2.0,2,27.0,58,7.0,7
25,Charles Leclerc,3.0,3,NaN,58,31.0,31
26,Lance Stroll,1.0,1,25.0,58,24.0,25
27,Lance Stroll,2.0,2,29.0,58,4.0,4



Step 3 is complete.


**sum of all stint_length_corrected = total laps**

In [ ]:
print("Starting Step 4: Final Verification...")

# --- Group by driver/race ---
group_cols = ['season', 'round', 'driver']
grouped = df2_cleaned.groupby(group_cols)

# --- Perform the verification check ---
# For each group, calculate the sum of our new stint lengths
stint_length_sum = grouped['stint_length_corrected'].sum()

# For each group, get the total laps (we use .first() because it's the same for all rows in a group)
total_laps = grouped['laps'].first()

# The check: compare the two series. Find any where the sum does not equal the total.
mismatched_groups = stint_length_sum[stint_length_sum != total_laps]

# --- Report the final result ---
if mismatched_groups.empty:
    print("\n✅🎉 CONGRATULATIONS! Final verification passed.")
    print("For every driver in every race, the sum of 'stint_length_corrected' now perfectly equals the total 'laps'.")
    print("Your data is clean, consistent, and ready for analysis!")
else:
    print("\n🚨 Verification Failed. Found inconsistencies in the following groups:")
    display(mismatched_groups)

print("\nCleaning process is complete.")

Starting Step 4: Final Verification...

✅🎉 CONGRATULATIONS! Final verification passed.
For every driver in every race, the sum of 'stint_length_corrected' now perfectly equals the total 'laps'.
Your data is clean, consistent, and ready for analysis!

Cleaning process is complete.


In [ ]:
print("Starting final cleanup process...")

# --- 1. Define columns to drop ---
# These are the original columns we've replaced and the temporary validation flags.
cols_to_drop = [
    'stint',
    'stint_length',
    'flag_row_count_ok',
    'flag_pit_lap_count_ok',
    'flag_pit_time_count_ok',
    'is_consistent'
]

# Drop the columns, using errors='ignore' in case a column was already removed.
df_final = df2_cleaned.drop(columns=cols_to_drop, errors='ignore')
print(f"Dropped {len(cols_to_drop)} old and temporary columns.")


# --- 2. Rename the corrected columns to their final names ---
rename_dict = {
    'stint_corrected': 'stint',
    'stint_length_corrected': 'stint_length'
}
df_final = df_final.rename(columns=rename_dict)
print("Renamed corrected columns to their final names.")

print("\nFinal DataFrame columns:")
print(df_final.columns)


# --- 3. Save the final, clean DataFrame to a new file ---
# We'll save it as a CSV file, which is a common and versatile format.
output_filename = 'f1_data_fully_cleaned.csv'
df_final.to_csv(output_filename, index=False)

print(f"\n✅ Success! Your fully cleaned data has been saved to '{output_filename}'.")
print("You can now use this file for all your future analysis.")

# --- Display a sample of the final, clean data ---
print("\n--- Sample of the final cleaned DataFrame ---")
display(df_final.head())

Starting final cleanup process...
Dropped 6 old and temporary columns.
Renamed corrected columns to their final names.

Final DataFrame columns:
Index(['season', 'round', 'circuit', 'driver', 'constructor', 'laps',
       'position', 'totalpitstops', 'avgpitstoptime', 'race_name',
       'time_of_race', 'location', 'country', 'air_temp_c', 'track_temp_c',
       'humidity_%', 'wind_speed_kmh', 'lap_time_variation',
       'tire_usage_aggression', 'fast_lap_attempts', 'position_changes',
       'driver_aggression_score', 'abbreviation', 'tire_compound', 'pit_lap',
       'pit_time', 'tire_compound_code', 'year', 'month', 'day', 'stint',
       'stint_length'],
      dtype='object')

✅ Success! Your fully cleaned data has been saved to 'f1_data_fully_cleaned.csv'.
You can now use this file for all your future analysis.

--- Sample of the final cleaned DataFrame ---


,season,round,circuit,driver,constructor,laps,position,totalpitstops,avgpitstoptime,race_name,...,abbreviation,tire_compound,pit_lap,pit_time,tire_compound_code,year,month,day,stint,stint_length
0,2018,1,Albert Park Grand Prix Circuit,Sebastian Vettel,Ferrari,58,1,1,21.787,Australian Grand Prix,...,VET,ULTRASOFT,26.0,21.787,C4,2018,3,25,1,26
1,2018,1,Albert Park Grand Prix Circuit,Sebastian Vettel,Ferrari,58,1,1,21.787,Australian Grand Prix,...,VET,SOFT,NaN,-1.000,C5,2018,3,25,2,32
2,2018,1,Albert Park Grand Prix Circuit,Lewis Hamilton,Mercedes,58,2,1,21.821,Australian Grand Prix,...,HAM,ULTRASOFT,19.0,21.821,C4,2018,3,25,1,19
3,2018,1,Albert Park Grand Prix Circuit,Lewis Hamilton,Mercedes,58,2,1,21.821,Australian Grand Prix,...,HAM,SOFT,NaN,-1.000,C5,2018,3,25,2,39
4,2018,1,Albert Park Grand Prix Circuit,Kimi Raikkönen,Ferrari,58,3,1,21.421,Australian Grand Prix,...,RAI,ULTRASOFT,18.0,21.421,C4,2018,3,25,1,18


In [ ]:
print("Preparing to save the final DataFrame to an Excel file...")

excel_output_filename = 'f1_data_fully_cleaned.xlsx'

try:
    df_final.to_excel(excel_output_filename, index=False, sheet_name='Cleaned F1 Data')

    print(f"\n Success! Your fully cleaned data has been saved to '{excel_output_filename}'.")

except Exception as e:
    print(f"\n An error occurred while saving the Excel file: {e}")
    print("Please make sure you have the 'openpyxl' library installed. You can install it by running: !pip install openpyxl")

Preparing to save the final DataFrame to an Excel file...

✅ Success! Your fully cleaned data has been saved to 'f1_data_fully_cleaned.xlsx'.


In [ ]:
print("Preparing to reorder columns...")

# --- 1. Define the new, logical order for your columns ---
new_column_order = [
    # Race Identifiers
    'season',
    'round',
    'race_name',
    'circuit',
    'location',
    'country',
    'year',
    'month',
    'day',
    'time_of_race',

    # Driver & Team
    'driver',
    'abbreviation',
    'constructor',

    # Race Performance
    'position',
    'laps',
    'totalpitstops',
    'position_changes',

    # Stint & Pit Stop Details
    'stint',
    'stint_length',
    'pit_lap',
    'pit_time',
    'avgpitstoptime',
    'tire_compound',
    'tire_compound_code',

    # Advanced Metrics
    'driver_aggression_score',
    'lap_time_variation',
    'tire_usage_aggression',
    'fast_lap_attempts',

    # Weather Conditions
    'air_temp_c',
    'track_temp_c',
    'humidity_%',
    'wind_speed_kmh'
]


# By passing the list to the DataFrame, pandas will rearrange the columns to match.
df_reordered = df_final[new_column_order]

print("✅ Successfully reordered the columns.")

# Display a sample of the new, reordered DataFrame
print("\n--- Sample of the reordered DataFrame ---")
display(df_reordered.head())


df_reordered.to_excel('f1_data_cleaned_reordered.xlsx', index=False, sheet_name='Cleaned F1 Data')

Preparing to reorder columns...
✅ Successfully reordered the columns.

--- Sample of the reordered DataFrame ---


,season,round,race_name,circuit,location,country,year,month,day,time_of_race,...,tire_compound,tire_compound_code,driver_aggression_score,lap_time_variation,tire_usage_aggression,fast_lap_attempts,air_temp_c,track_temp_c,humidity_%,wind_speed_kmh
0,2018,1,Australian Grand Prix,Albert Park Grand Prix Circuit,Melbourne,Australia,2018,3,25,05:10:00,...,ULTRASOFT,C4,6.755003,0.001723,0.017241,44.76882,15.783333,22.283333,57.0,23.8
1,2018,1,Australian Grand Prix,Albert Park Grand Prix Circuit,Melbourne,Australia,2018,3,25,05:10:00,...,SOFT,C5,6.755003,0.001723,0.017241,44.76882,15.783333,22.283333,57.0,23.8
2,2018,1,Australian Grand Prix,Albert Park Grand Prix Circuit,Melbourne,Australia,2018,3,25,05:10:00,...,ULTRASOFT,C4,6.754254,0.001735,0.017241,44.73482,15.783333,22.283333,57.0,23.8
3,2018,1,Australian Grand Prix,Albert Park Grand Prix Circuit,Melbourne,Australia,2018,3,25,05:10:00,...,SOFT,C5,6.754254,0.001735,0.017241,44.73482,15.783333,22.283333,57.0,23.8
4,2018,1,Australian Grand Prix,Albert Park Grand Prix Circuit,Melbourne,Australia,2018,3,25,05:10:00,...,ULTRASOFT,C4,6.818562,0.001603,0.017241,45.13482,15.783333,22.283333,57.0,23.8
